In [ ]:
import os.path as osp
import sys
import matplotlib.pyplot as plt
import numpy as np

sys.path.append("/home/viktorv/Projects/3DMNN/main/models/latent_space/src")

from utils.templates import mlp_architecture_ala_iclr_18, default_train_params
from classes.autoencoder import Configuration as Conf
from classes.pointnet_ae import PointNetAutoEncoder
from utils.io import obj_wrapper

from utils.io import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from utils.utils import reset_tf_graph
from classes.latent_gan import LatentGAN

top_out_dir = '/home/viktorv/Projects/3DMNN/data/point_cloud_sampled/data'          # Use to save Neural-Net check-points etc.
top_in_dir = '/home/viktorv/Projects/3DMNN/data/point_cloud_sampled/data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.

experiment_name = 'single_class_ae'
n_pc_points = 2048                # Number of points per model.
bneck_size = 128                  # Bottleneck-AE size
ae_loss = 'emd'                   # Loss to optimize: 'emd' or 'chamfer'

class_name = input('Give me the class name (e.g. "chair"): ').lower()

syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir, syn_id)

all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

train_params = default_train_params()

encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)
train_dir = create_dir(osp.join(top_out_dir, experiment_name))

conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = 50,
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5   # How often to evaluate/print out loss on 
                         # held_out data (if they are provided in ae.train() ).
conf.save(osp.join(train_dir, 'configuration'))

reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

buf_size = 1 # Make 'training_stats' file to flush each output line regarding training.
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(all_pc_data, conf, log_file=fout)
fout.close()


Give me the class name (e.g. "chair"): car
<generator object files_in_subdirs at 0x7fefdf6e4678>
7497 pclouds were loaded. They belong in 1 shape-classes.
Building Encoder
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
encoder_conv_layer_0 conv params =  256
bnorm params =  128
Tensor("single_class_ae_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_1 conv params =  8320
bnorm params =  256
Tensor("single_class_ae_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_2 conv params =  16512
bnorm params =  256
Tensor("single_class_ae_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  33024
bnorm params =  512
Tensor("single_class_ae_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  32896
bnorm params =  256
Tensor("single

In [ ]:
feed_pc, feed_model_names, _ = all_pc_data.full_epoch_data()
reconstructions = np.asarray(ae.reconstruct(feed_pc))
latent_codes = ae.transform(feed_pc)

def reconstruct_obj(reconstructions):
    
    for i, reconstruction in enumerate(reconstructions[0]):
        obj_wrapper(reconstruction, class_name, i)
        
    print("Reconstructed")
    print(reconstructions[0].shape)

def interpolate(_from, _to, steps=10):
    
    print("Generating interpolations")
    interpolations = ae.interpolate(reconstructions[0][_from], reconstructions[0][_to], steps)

    for i, interpolation in enumerate(interpolations):
        obj_wrapper(interpolation, class_name + "_intr", i)

    print("Interpolation Complete")
    print(interpolations.shape)

In [ ]:
    
discriminator = None 
generator = None


print(latent_codes.shape)
# TODO:
# 1. Rewrite CONF Class to fit GANs
# 2. Implement Discriminator, Generator for the simple LGAN
# 3. Train
# 4. Try to decode something generated
# 5. Implement Conditional GAN in the LGAN
# 6. Go to 3 and 4 again


lgan_conf = {
    "name": "lgan",
    learning_rate: 10e-5,
    n_output: (1,128),
    noise_dim: (1, 64),
    discriminator: discriminator,
    generator: generator,
    beta: 0.9,
}

lgan = LatentGAN(lgan_conf)

# Takes
# (self, name, learning_rate, n_output, noise_dim, discriminator, generator, beta=0.9, gen_kwargs={}, disc_kwargs={}, graph=None)

lgan_train_conf = {
    training_epochs: 50,
    batch_size: 50,
    loss_display_step: 1,
    train_dir: "./TrainTest",
    saver_step: 10,
    summary_step: 20,
}

train_stats_lgan = lgan.train(latent_codes, lgan_train_conf)